<a href="https://colab.research.google.com/github/The-DigitalAcademy/TrendHunt-Application/blob/Model/Model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 22.0 MB/s eta 0:00:00


In [34]:
%%writefile app.py

import streamlit as st
import os
import keras
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from PIL import Image
import pickle
from scipy.spatial import distance
import pandas as pd

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Load the saved PCA features and images list
file_path = '/content/my_data.p'
images, pca_features, pca = pickle.load(open(file_path, 'rb'))

# Define the feature extractor using the VGG16 model
base_model = VGG16(weights='imagenet')
layer_name = 'fc2'
feat_extractor = keras.models.Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)

# Function to load and preprocess an image
def load_image(path):
    img = image.load_img(path, target_size=(224, 224))

    # Ensure the image is in RGB mode
    if img.mode != "RGB":
        img = img.convert("RGB")

    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    return x

# Streamlit UI
st.title("Trendhunt Application")

# Landing page (About and Login)
landing_page = st.sidebar.selectbox("Select Page", ["About", "Login"])

if landing_page == "About":
    st.write("The Trendhunt app is a revolutionary solution designed to address the challenges people face in locating specific items they desire in today's digital age. With the abundance of information on the internet, finding the exact product can be a daunting task. Trendhunt aims to simplify this process by providing a comprehensive and easily accessible platform for users to discover similar items based on a picture they upload.")

    # Add a picture under the "Trendhunt application" text
    st.sidebar.image("https://i.fbcd.co/products/resized/resized-750-500/ddc5250f28dcf85d8238ecc732a8752b73e3685762d6dad4b32c4c9359538e77.jpg", caption="Trendhunt Logo", use_column_width=True)

elif landing_page == "Login":
    st.sidebar.write("Login Page")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    login_button = st.button("Login")

    if login_button:
        # Check login credentials (you can replace this with your authentication logic)
        if username == "your_username" and password == "your_password":
            st.success("Login successful! Redirecting to the upload page...")
            landing_page = "Upload Image"  # Redirect to the upload page
        else:
            st.error("Login failed. Please check your credentials.")

# Upload an image
uploaded_image = st.file_uploader("Upload an image...", type=["jpg", "png", "jpeg"])

if uploaded_image is not None:
    # Display the uploaded image
    st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

    # Perform predictions on the uploaded image
    img = Image.open(uploaded_image)

    # Convert the image to RGB mode explicitly
    img = img.convert("RGB")

    img_path = "temp_image.jpg"
    img.save(img_path)

    # Load and preprocess the uploaded image
    x = load_image(img_path)

    # Make predictions using the model
    predictions = model.predict(x)

    # Decode the predictions to get class labels and probabilities
    decoded_predictions = decode_predictions(predictions, top=5)[0]


    # Calculate PCA features for the uploaded image
    img_features = feat_extractor.predict(x)[0]
    img_pca_features = pca.transform([img_features])[0]

    # Calculate distances to all other images
    distances = [distance.cosine(img_pca_features, feat) for feat in pca_features]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[0:5]

    st.subheader("Most Similar Images:")

    # Read the data from the CSV file once
    data = pd.read_csv("styles.csv", encoding='utf-8', error_bad_lines=False, warn_bad_lines=True)

    for idx in idx_closest:
        similar_img = Image.open(images[idx])
        product_id = os.path.splitext(os.path.basename(images[idx]))[0]
        caption = data[data['id'] == int(product_id)]['productDisplayName'].values[0]
        caption1 = data[data['id'] == int(product_id)]['gender'].values[0]

        st.image(similar_img, use_column_width=True)
        lst = [caption, caption1]

        lst = [f'Product Name: {caption}' , f'Gender: {caption1}']
        for i in lst:
            st.markdown(i)



Overwriting app.py


In [35]:
!wget -q -O - ipv4.icanhazip.com

35.225.105.137


In [36]:
!streamlit run app.py & npx localtunnel --port 8501

[#.................] - fetchMetadata: sill resolveWithNewModule follow-redirect


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.225.105.137:8501

npx: installed 22 in 4.932s
your url is: https://whole-pugs-deny.loca.lt
2023-09-20 14:25:32.149263: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 14:25:33.927769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 1s 823ms/step
/content/app.py:101: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("styles.csv", encoding='utf-8', erro